<a href="https://colab.research.google.com/github/Matrix7043/Machine_learning101/blob/main/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from typing import Callable
import numpy as np
from numpy import ndarray
import pandas as pd

In [39]:
df = pd.read_csv('/content/sample_data/diamond.csv')
df1 = df[['carat', 'x', 'y', 'z', 'depth']].head(100)
df2 = df[['price']].head(100)
y = np.array(df2['price'])
x = np.transpose(np.array([df1['carat'],df1['depth'], df1['x'], df1['y'], df1['z']]))

In [63]:
Array_function = Callable[[ndarray, ndarray], ndarray]
Chain = list[Array_function]

In [40]:
def init_weights(X: ndarray,
                 zeros: bool = False,
                 ones: bool = False) -> dict[str, ndarray]:

    weights: dict[str, ndarray] = {}

    if ones:
        weights['W'] = np.ones(X.shape[1])
        weights['B'] = np.ones(X.shape[0])
    elif zeros:
        weights['W'] = np.zeros(X.shape[1])
        weights['B'] = np.zeros(X.shape[0])
    else:
        weights['W'] = np.random.randn(X.shape[1])
        weights['B'] = np.random.randn(X.shape[0])

    return weights

In [64]:
def deriv2(input: ndarray,
          input_nd: ndarray,
          function: Array_function,
          delta: float = 0.001) -> ndarray:

    return (function(input + delta, input_nd) - function(input - delta, input_nd))/(2*delta)

In [71]:
def deriv(func: Callable[[ndarray], ndarray],
          input_: ndarray,
          delta: float = 0.001) -> ndarray:

    return (func(input_ + delta) - func(input_ - delta)) / (2 * delta)

In [57]:
def mse(P: ndarray,
        Y: ndarray) -> ndarray:

    return np.power((Y - P), 2)


In [66]:
def sigmoid(x: ndarray) -> ndarray:

    return 1 / (1 + np.exp(-x))


In [72]:
def chain_deriv(chain: Chain,
                input_range: ndarray) -> ndarray:

    argument: list[ndarray] = [input_range]
    for j, i in enumerate(chain):
        argument.append(i(argument[j]))

    answer = 1
    argument.pop()
    for i, j in zip(chain, argument):
        answer *= deriv(i, j)

    return answer


In [76]:
def chain_deriv_2(chain: Chain,
                  input_range: ndarray) -> ndarray:
    '''
    Uses the chain rule to compute the derivative of two nested functions:
    (f2(f1(x))' = f2'(f1(x)) * f1'(x)
    '''
    assert len(chain) == 2, \
    "This function requires 'Chain' objects of length 2"
    assert input_range.ndim == 1, \
    "Function requires a 1 dimensional ndarray as input_range"
    f1 = chain[0]
    f2 = chain[1]
    # df1/dx
    f1_of_x = f1(input_range)
    # df1/du
    df1dx = deriv(f1, input_range)
    # df2/du(f1(x))
    df2du = deriv(f2, f1(input_range))
    # Multiplying these quantities together at each point
    return df1dx * df2du


In [77]:
x = np.random.randn(5)
print(chain_deriv([sigmoid, sigmoid], x))
print(chain_deriv_2([sigmoid, sigmoid], x))

[0.03032495 0.01355186 0.0547875  0.05534    0.05348701]
[0.03032495 0.01355186 0.0547875  0.05534    0.05348701]


In [41]:
def forward_pass(x_batch: ndarray,
                 y_batch: ndarray,
                 weights: dict[str, ndarray]) -> dict[str, ndarray]:

                 assert x_batch.shape[0] == y_batch.shape[0]

                 assert x_batch.shape[1] == weights['W'].shape[0]

                 assert weights['B'].shape[0] == y_batch.shape[0]

                 W = weights['W']

                 N = np.dot(x_batch, W)

                 P = N + weights['B']

                 L = np.mean(np.power((y_batch - P), 2))

                 forward_info: dict[str, ndarray] = {}
                 forward_info['X'] = x_batch
                 forward_info['Y'] = y_batch
                 forward_info['N'] = N
                 forward_info['P'] = P

                 return forward_info, L


In [42]:
def back_propagation(weights: dict[str, ndarray],
                     forward_info: dict[str, ndarray]) -> dict[str, ndarray]:

                     dLdP = -2 * (forward_info['Y'] - forward_info['P'])

                     dPdN = np.ones_like(forward_info['N'])

                     dLdN = dLdP * dPdN

                     dPdB = np.ones_like(weights['B'])

                     dNdW = np.transpose(forward_info['X'])

                     dLdW = np.dot(dNdW, dLdN)

                     dLdB = (dLdP * dPdB).sum(axis=0)

                     loss_gradient: dict[str, ndarray] = {}
                     loss_gradient['W'] = dLdW
                     loss_gradient['B'] = dLdB

                     return loss_gradient


In [48]:
def train(X_batch: ndarray,
          Y_batch: ndarray,
          iteration: float = 20000,
          learning_rate = 0.000001) -> ndarray:

          weights = init_weights(X_batch)

          for i in range(iteration):

            forward_info, loss = forward_pass(X_batch, Y_batch, weights)

            if i%10 == 0:
                print(loss)

            loss_gradient = back_propagation(weights, forward_info)

            for key in weights.keys():
              weights[key] -= learning_rate * loss_gradient[key]

          return weights

In [44]:
def predict(X: ndarray,
            weights: dict[str, ndarray]):

    N = np.dot(X, weights['W'])
    return N + weights['B'][:10]

In [49]:
theta = train(x, y)

944924.69226698
492261.54224101076
490950.54062713706
489643.68320744194
488340.956876574
487042.3485707073
485747.8452673275
484457.4339851026
483171.10178375134
481888.83576391416
480610.62306702323
479336.4508751743
478066.30641099723
476800.176937528
475538.04975808156
474279.9122161238
473025.75169514434
471775.5556185312
470529.3114494431
469287.006690685
468048.62888458156
466814.1656128531
465583.6044964907
464356.93319563166
463134.13940943626
461915.2108759644
460700.13537205145
459488.9007131873
458281.494753393
457077.9053850992
455878.12053902476
454682.12818405556
453489.9163271237
452301.47301308764
451116.786324612
449935.8443820474
448758.6353433125
447585.1474037745
446415.3687961301
445249.2877902889
444086.8926932546
442928.1718490079
441773.1136383901
440621.70647898555
439473.93882500665
438329.79916717694
437189.2760326166
436052.3579847262
434919.0336230733
433789.291583277
432663.12053689436
431540.50919130683
430421.4462896071
429305.9206104854
428193.92096811